# one-hot表示词向量（基于numpy实现）

In [1]:
import jieba

data = ["我爱我的祖国",'我喜欢祖国的大好河山']
samples = [' '.join(jieba.cut(doc)) for doc in data]
print(samples)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\priv_\AppData\Local\Temp\jieba.cache
Loading model cost 0.436 seconds.
Prefix dict has been built successfully.


['我 爱 我 的 祖国', '我 喜欢 祖国 的 大好河山']


In [2]:
# 词级别的one-hot编码
import numpy as np

token_index = {}  # 构造一个空的索引集合
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index)  # 给每个唯一单词指定一个唯一索引
max_length = len(token_index)  # 对样本进行分词，只考虑每个样本前max-length个单词
results = np.zeros(shape=(len(samples), max_length, max(token_index.values())+1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1
print(token_index)
print(results)

{'我': 0, '爱': 1, '的': 2, '祖国': 3, '喜欢': 4, '大好河山': 5}
[[[1. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0.]]]


# Bag of words表示词向量（基于sklearn的实现）

In [3]:
samples

['我 爱 我 的 祖国', '我 喜欢 祖国 的 大好河山']

In [4]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b', binary=False)
X = vectorizer.fit_transform(samples)
print(vectorizer.get_feature_names())
print(X.toarray())

['喜欢', '大好河山', '我', '爱', '的', '祖国']
[[0 0 2 1 1 1]
 [1 1 1 0 1 1]]


C:\Users\priv_\.conda\envs\py37_torch18_cuda111\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# TF_IDF表示词向量（基于sklearn实现）

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
X = vectorizer.fit_transform(samples)
print(vectorizer.get_feature_names())
print(X)

['喜欢', '大好河山', '我', '爱', '的', '祖国']
  (0, 5)	0.3540997415957358
  (0, 4)	0.3540997415957358
  (0, 3)	0.4976748316029239
  (0, 2)	0.7081994831914716
  (1, 1)	0.5330978245262535
  (1, 0)	0.5330978245262535
  (1, 5)	0.3793034928087496
  (1, 4)	0.3793034928087496
  (1, 2)	0.3793034928087496


In [7]:
print(X.toarray())

[[0.         0.         0.70819948 0.49767483 0.35409974 0.35409974]
 [0.53309782 0.53309782 0.37930349 0.         0.37930349 0.37930349]]


# 代码实现TF_IDF计算过程

In [12]:
import numpy as np
from collections import Counter

class TF_IDF_Model(object):
    def __init__(self, documents_list):
        # 文本列表
        self.documents_list = documents_list
        # 文本总个数
        self.documents_number = len(documents_list)
        # 存储每个文本中每个词的词频
        self.tf = []
        # 存储每个词汇的逆文档频率
        self.idf = {}
        # 类初始化
        self.init()
    
    # 初始化，计算每个文本中的词频和逆文档频率
    def init(self):
        # 每个词汇的文档频率
        df = {}
        # 遍历每个文本列表中的每个文本
        for document in self.documents_list:
            temp = {}
            # 遍历每个文本中的每个词
            for word in document:
                # 存储每个文本中每个词的词频
                temp[word] = temp.get(word, 0) + 1/len(document)
            # 遍历完一个文本，就将该文本计算的词频以字典的形式存储到tf列表中
            self.tf.append(temp)
            # 遍历temp中的词，每出现一次该词的文档频率就加一，计算该词的文本数
            for key in temp.keys():
                df[key] = df.get(key, 0) + 1
        # 计算每个词的逆文档频率
        for key, value in df.items():
            self.idf[key] = np.log(self.documents_number / (value + 1))
    
    # 计算查询文本与文本列表中某个文本的tf-idf相似值
    def get_score(self, index, query):
        score = 0.0
        # 遍历查询文本的每个词
        for q in query:
            # 判断词是否在文本中，如果没有就继续查下一个
            if q not in self.tf[index]:
                continue
            # 如果有，就将该词的tf-idf值加到score中
            score += self.tf[index][q] * self.idf[q]
        # 返回最终的得分
        return score
    
    # 计算查询文本与文本列表中每个文本的tf-idf相似值
    def get_documents_score(self, query):
        score_list = []
        for i in range(self.documents_number):
            score_list.append(self.get_score(i, query))
        return score_list

In [13]:
document_list = ["行政机关强行解除行政协议造成损失，如何索取赔偿？",
                 "借钱给朋友到期不还得什么时候可以起诉？怎么起诉？",
                 "我在微信上被骗了，请问被骗多少钱才可以立案？",
                 "公民对于选举委员会对选民的资格申诉的处理决定不服，能不能去法院起诉吗？",
                 "有人走私两万元，怎么处置他？",
                 "法律上餐具、饮具集中消毒服务单位的责任是不是对消毒餐具、饮具进行检验？"]

import jieba
document_list = [list(jieba.cut(doc)) for doc in document_list]

tf_idf_model = TF_IDF_Model(document_list)

print(tf_idf_model.tf)
print(tf_idf_model.idf)

query = "走私了两万元，在法律上应该怎么量刑？"
query = list(jieba.cut(query))
scores = tf_idf_model.get_documents_score(query)
print(scores)

[{'行政': 0.15384615384615385, '机关': 0.07692307692307693, '强行': 0.07692307692307693, '解除': 0.07692307692307693, '协议': 0.07692307692307693, '造成': 0.07692307692307693, '损失': 0.07692307692307693, '，': 0.07692307692307693, '如何': 0.07692307692307693, '索取': 0.07692307692307693, '赔偿': 0.07692307692307693, '？': 0.07692307692307693}, {'借钱': 0.06666666666666667, '给': 0.06666666666666667, '朋友': 0.06666666666666667, '到期': 0.06666666666666667, '不': 0.06666666666666667, '还': 0.06666666666666667, '得': 0.06666666666666667, '什么': 0.06666666666666667, '时候': 0.06666666666666667, '可以': 0.06666666666666667, '起诉': 0.13333333333333333, '？': 0.13333333333333333, '怎么': 0.06666666666666667}, {'我': 0.058823529411764705, '在': 0.058823529411764705, '微信': 0.058823529411764705, '上': 0.058823529411764705, '被': 0.11764705882352941, '骗': 0.11764705882352941, '了': 0.058823529411764705, '，': 0.058823529411764705, '请问': 0.058823529411764705, '多少': 0.058823529411764705, '钱': 0.058823529411764705, '才': 0.058823529411764705, '

# TF_IDF关键词提取（基于jieba）


- import jieba.analyse

* jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
    * sentence 为待提取的文本
    * topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
    * withWeight 为是否一并返回关键词权重值，默认值为 False
    * allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [14]:
import jieba.analyse as analyse

with open('NBA.txt', encoding='utf-8') as f:
    lines = f.read()
print("  ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

韦少  杜兰特  全明星  全明星赛  MVP  威少  正赛  科尔  投篮  勇士  球员  斯布鲁克  更衣柜  NBA  三连庄  张卫平  西部  指导  雷霆  明星队


# TextRank关键词提取（基于jieba）

* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
* jieba.analyse.TextRank() 新建自定义 TextRank 实例

* 算法论文： [TextRank: Bringing Order into Texts](http://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

* 基本思想:
    * 将待抽取关键词的文本进行分词
    * 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
    * 计算图中节点的PageRank，注意是无向带权图
* 阅读资料：
    * [基于textrank的关键词抽取方法](https://blog.csdn.net/zhangf666/article/details/77841845)
    * [pagerank算法核心思想](https://www.jianshu.com/p/f6d66ab97332)
    * [浅析PageRank算法](http://blog.codinglabs.org/articles/intro-to-pagerank.html)

In [15]:
import jieba.analyse as analyse

with open('NBA.txt', encoding='utf-8') as f:
    lines = f.read()
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------分割线----------------")
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

全明星赛  勇士  正赛  指导  对方  投篮  球员  没有  出现  时间  威少  认为  看来  结果  相隔  助攻  现场  三连庄  介绍  嘉宾
---------------------分割线----------------
勇士  正赛  全明星赛  指导  投篮  玩命  时间  对方  现场  结果  球员  嘉宾  时候  全队  主持人  照片  全程  目标  快船队  肥皂剧


# glove加载并表示词向量（使用gensim加载）

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# 输入文件
glove_file = './glove.6B.100d.txt'
# 输出文件
tmp_file = "glove2word2vec.txt"

# 转换
(count, dimensions) = glove2word2vec(glove_file, tmp_file)

print(count)
print(dimensions)

In [ ]:
# 加载转换后的文件
model = KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
# 获取某个单词的向量表示，直接以下标方式访问即可
cat_vec = model['cat']
print(cat_vec)
# 获得单词cat的最相似向量的词汇
print(model.most_similar('cat'))

# fasttext生成词向量（使用gensim）

In [20]:
from gensim.models import FastText

print(samples)
fasttext_model = FastText(samples,  vector_size=20, window=3, min_count=1, epochs=10, min_n=3 , max_n=6,word_ngrams=1)

['我 爱 我 的 祖国', '我 喜欢 祖国 的 大好河山']


In [22]:
print(fasttext_model.wv['我'])

[ 0.02302497  0.00666335  0.03425618 -0.00233102  0.03433145 -0.01923007
 -0.01505522  0.02630648 -0.03774588 -0.01843461 -0.00756334 -0.00232549
 -0.00049289 -0.02973611  0.00931649 -0.00748862  0.01399848 -0.03564914
 -0.0082388  -0.03114243]
